In [10]:
import numpy as np

In [11]:
x = np.array([1,2,3,5,10])

In [12]:
def forward_predictor(data,window_width):
    """
    Calculate the forward prediction based on data. https://doi.org/10.1016/0165-0270(91)90118-J
    """
    output = [data[0]] # Leere Liste wird erstellt, da wir diese nach und nach befüllen.
    for i in range(1,len(data)): # range gibt uns nach und nach die Indices [0, N-1], wo N die Länge der Daten ist. 
        if i-window_width>=0: 
            prediction = 1/window_width*np.sum(data[i-window_width:i])
        else:
            prediction = 1/i*np.sum(data[:i])
        output.append(prediction)
    return output

In [13]:
#tests
x1 = np.ones(5) # erstellt array der nur einsen enthält. 
print(forward_predictor(x1,4))
print(forward_predictor(x,4))

[1.0, 1.0, 1.0, 1.0, 1.0]
[1, 1.0, 1.5, 2.0, 2.75]


In [14]:
def backward_predictor(data,window_width):
    """
    Calculate the backward prediction based on data. https://doi.org/10.1016/0165-0270(91)90118-J
    """
    output = list()
    len_data = len(data)
    for i in range(len_data-1):
        if i+window_width<len_data: 
            prediction = 1/window_width*np.sum(data[i+1:i+1+window_width])
        else:
            prediction = 1/(len_data-i-1)*np.sum(data[i+1:])
        output.append(prediction)
    output.append(data[-1])
    return output

In [15]:
print(backward_predictor(x1,4))
print(backward_predictor(x,3))

[1.0, 1.0, 1.0, 1.0, 1.0]
[3.333333333333333, 6.0, 7.5, 10.0, 10]


In [49]:
def calculate_forward_weights(data, M, forward_prediction, p):
    """
    Calculate the weight of the forward predictor.
    """
    #predictors should be given as lists, if the first element of forward_prediction is not a list
    #it is assumed that only a single forward prediction was given
    if type(forward_prediction[0]) is not list:
        forward_prediction=[forward_prediction]
    
    len_data = len(data)
    n_predictors = len(forward_prediction)
    #allocate weight array
    f = np.zeros((n_predictors,len_data))

    for i, prediction in enumerate(forward_prediction):
        for k in range(len_data):
            for j in range(M):
                if not k-j<0:
                    f[i,k] += (data[k-j]-prediction[k-j])**2
            #zero should only occur at the beginning, in order to avoid infinities it is replaced by one
            if f[i,k]==0: f[i,k]=1
            f[i,k] = f[i,k]**-p
    return f

In [35]:
pr = [forward_predictor(x,2),forward_predictor(x,3)]

In [42]:
pr[0][0]

1

In [45]:
type(pr[0][0]) is not list

True

In [50]:
calculate_forward_weights(x,2,pr[0],1)

5


array([[ 1.        ,  1.        ,  0.30769231,  0.11764706,  0.02366864]])

In [28]:
((5-2.5)**2+2.25)**-1

0.11764705882352941

In [30]:
((6)**2+6.25)**-1

0.023668639053254437

In [56]:
def calculate_backward_weights(data, M, backward_prediction, p):
    """
    Calculate the weight of the forward predictor.
    """
    #predictors should be given as lists, if the first element of backward_prediction is not a list
    #it is assumed that only a single backward prediction was given
    if type(backward_prediction[0]) is not list:
        backward_prediction=[backward_prediction]
    
    len_data = len(data)
    n_predictors = len(backward_prediction)
    #allocate weight array
    b = np.zeros((n_predictors,len_data))

    for i, prediction in enumerate(backward_prediction):
        for k in range(len_data):
            for j in range(M):
                if not k+j>=len_data:
                    b[i,k] += (data[k+j]-prediction[k+j])**2
            #zero should only occur at the beginning, in order to avoid infinities it is replaced by one
            if b[i,k]==0: b[i,k]=1
            b[i,k] = b[i,k]**-p
    return b

In [57]:
bpr = [backward_predictor(x,2),backward_predictor(x,3)]

In [58]:
calculate_backward_weights(x,2,bpr[0],1)

array([[ 0.16      ,  0.04123711,  0.02209945,  0.04      ,  1.        ]])

In [59]:
bpr

[[2.5, 4.0, 7.5, 10.0, 10], [3.333333333333333, 6.0, 7.5, 10.0, 10]]

In [60]:
x

array([ 1,  2,  3,  5, 10])

In [61]:
1/6.25

0.16